**Install packages and download dataset**

In [ ]:
# Download kornia and other packages which are not available natively on colab
!pip install gdown kornia pytorch-lightning==1.8.3.post1

In [ ]:
# Download dataset
!gdown --fuzzy https://drive.google.com/file/d/1VSIMAR3-2fXTEy-QdY2d0M_-aC1aXfWp/view
!unzip Classification_data.zip

In [3]:
# main imports
import os
import cv2 as cv
import random
import numpy as np
from pathlib import Path
import torch
import math
from torchvision.transforms.functional import resize, gaussian_blur, hflip
from kornia.color import hls_to_rgb, rgb_to_hls
from math import pi, cos
from kornia.augmentation import ColorJiggle, RandomPlanckianJitter, RandomGaussianBlur
from kornia.geometry import get_perspective_transform, warp_perspective, transform_points
from torch.utils.data import Dataset
from torchvision import io as tvio
from torch import nn
import torch.nn.functional as F
from pytorch_lightning import LightningModule, Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.optim.lr_scheduler import _LRScheduler

In [4]:
torch.manual_seed(42)

In [5]:
# Get image names and image labels in the list
def get_image_names_and_labels(split):
  image_names_all = []
  image_labels_all = []

  for folder in os.listdir(Path("Classification_data") / split):
    image_labels = []
    image_names = os.listdir(Path("Classification_data") / split / folder)
    image_labels.extend([folder] * len(image_names))
    image_names_all.extend(image_names)
    image_labels_all.extend(image_labels) 
  return image_names_all, image_labels_all 


In [9]:
# Image names and labels and label_to_index for training
image_names, image_labels = get_image_names_and_labels("train")
image_names_test, image_labels_test = get_image_names_and_labels("test")
# dict containing label(str) and index(int)
labels_dict = {label:index for index, label in enumerate(set(image_labels))}

In [10]:
# Augmentations
class BlurImage:
    def __call__(self, img, blur_bool):
        if blur_bool:
            kernel_size_int = random.randrange(3, 11, 2)
            transformed = gaussian_blur(img, kernel_size=(kernel_size_int, kernel_size_int), sigma=(0.1, 2.0))
            return transformed
        else:
            return img

class ColorAugs:
    def __init__(self, shifts=(0.0, 0.2)):
        self.color_aug_func = {
            0: self.no_color_aug,
            1: self.isonoise,
            2: self.color_jiggle,
            3: self.random_planckian_jitter,
        }
        self.brightness_shift = random.uniform(*shifts)
        self.contrast_shift = random.uniform(*shifts)
        self.saturation_shift = random.uniform(*shifts)
        self.hue_shift = random.uniform(*shifts)
        self.color_jiggle = ColorJiggle(self.brightness_shift, self.contrast_shift, self.saturation_shift,
                                        self.hue_shift, p=1.)
        self.random_planckian_jitter = RandomPlanckianJitter(mode="blackbody", select_from=[6, 12, 18, 24])

    def no_color_aug(self, img):
        return img

    def isonoise(self, img, color_shift=(0.01, 0.2), intensity=(0.1, 0.6)):
        img = img.unsqueeze(0)
        color_shift = random.uniform(*color_shift)
        intensity = random.uniform(*intensity)
        hls_img = rgb_to_hls(img)
        l_std = hls_img[:, 1].flatten(1).std(-1)
        luminance_noise = torch.poisson(
            (l_std * intensity * 255)[:, None, None].expand(-1, img.size(-2), img.size(-1))
        )
        color_noise = torch.empty(img.shape[-2:]).normal_(
            0, color_shift * 2 * pi * intensity
        )
        hls_img[:, 0] += color_noise[None]
        hls_img[:, 0] %= 2 * pi
        hls_img[:, 1] += (luminance_noise / 255) * (1.0 - hls_img[:, 1])
        return hls_to_rgb(hls_img).squeeze(0)

    def color_jiggle(self, img):
        transformed = self.color_jiggle(img)
        return transformed[0]

    def random_planckian_jitter(self, img):
        transformed = self.random_planckian_jitter(img)
        return transformed[0]


class TransformImageAngle:
    def __call__(self, image, angle, img_width, img_height):
        angle_rad = (angle / 180) * math.pi
        coordinates = torch.Tensor([
            [0, 1],
            [1, 1],
            [1, 0],
            [0, 0]
        ])
        sin, cos = math.sin(angle_rad), math.cos(angle_rad)
        rot_mat = torch.Tensor([cos, -sin, sin, cos]).view(2, 2)
        target_coordinates = coordinates @ rot_mat
        target_coordinates = target_coordinates.unsqueeze(0)
        M = get_perspective_transform(coordinates.unsqueeze(0), target_coordinates)
        image_warped = warp_perspective(image.float().unsqueeze(0), M, dsize=(img_height, img_width))
        return image_warped.squeeze(0)


In [11]:
# Train test split 
from sklearn.model_selection import train_test_split

image_names_train, image_names_validation, image_labels_train, image_labels_validation = train_test_split(
    image_names, image_labels, test_size=0.2, random_state=42, stratify=image_labels
)

In [13]:
# Datasets(Train/Val/Test)
class ImageDataset(Dataset):
    image_dir = Path("Classification_data/train")

    def __init__(self, resize_img_height, resize_img_width, image_names_train, image_labels_train, label_index_dict):
        super().__init__()
        self.hflip = hflip
        self.transform_image_angle = TransformImageAngle()
        self.color_augs = ColorAugs()
        self.blur = BlurImage()
        self.resize_img_height = resize_img_height
        self.resize_img_width = resize_img_width
        self.image_names_train = image_names_train
        self.image_labels_train = image_labels_train
        self.label_index_dict = label_index_dict

    def read_image(self, image_id):
        image = tvio.read_image(str(self.image_dir / self.image_labels_train[image_id] / self.image_names_train[image_id])) / 255
        image = resize(image, (self.resize_img_height, self.resize_img_width))
        return image

    def __len__(self):
        return len(self.image_names_train)

    def __getitem__(self, idx: int):
        image = self.read_image(idx)

        # Hflip image targets
        horizontal_flip = torch.randint(0, 2, (1,)).item()
        if horizontal_flip:
          image = self.hflip(image)
        # Rotate
        angle = torch.randint(-5, 6, size=(1,)).item()
        image = self.transform_image_angle(image, angle, self.resize_img_height, self.resize_img_width)
        # Color augmentations/noise
        aug_color_func_index = torch.randint(0, 4, (1,)).item()
        col_aug_func = self.color_augs.color_aug_func[aug_color_func_index]
        image = col_aug_func(image)
        # Blur
        blur_bool = torch.randint(0, 2, (1,)).item()
        image = self.blur(image, blur_bool)
        return image, self.label_index_dict[self.image_labels_train[idx]]


class ImageDatasetValidation(ImageDataset):
  image_dir = Path("Classification_data/train")

  def __init__(self, resize_img_height, resize_img_width, image_names_validation, image_labels_validation, label_index_dict):
    super().__init__(resize_img_height, resize_img_width, image_names_validation, image_labels_validation, label_index_dict)
    self.image_names_validation = image_names_validation
    self.image_labels_validation = image_labels_validation
  
  def read_image(self, image_id):
      image = tvio.read_image(str(self.image_dir / self.image_labels_validation[image_id] / self.image_names_validation[image_id])) / 255
      image = resize(image, (self.resize_img_height, self.resize_img_width))
      return image

  def __len__(self):
    return len(self.image_names_validation)

  def __getitem__(self, idx: int):
    image = self.read_image(idx)
    return image, self.label_index_dict[self.image_labels_validation[idx]]


class ImageDatasetTest(ImageDataset):
  image_dir = Path("Classification_data/test")

  def __init__(self, resize_img_height, resize_img_width, image_names_test, image_labels_test, label_index_dict):
    super().__init__(resize_img_height, resize_img_width, image_names_test, image_labels_test, label_index_dict)
    self.image_names_test = image_names_test
    self.image_labels_test = image_labels_test
  
  def read_image(self, image_id):
        image = tvio.read_image(str(self.image_dir / self.image_labels_test[image_id] / self.image_names_test[image_id])) / 255
        image = resize(image, (self.resize_img_height, self.resize_img_width))
        return image

  def __len__(self):
    return len(self.image_names_test)

  def __getitem__(self, idx: int):
    image = self.read_image(idx)
    return image, self.label_index_dict[self.image_labels_test[idx]]

In [14]:
# Some basic small conv network 
class ConvNetwork(nn.Module):
  def __init__(self, stride, num_channels, num_classes):
    super().__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=num_channels, kernel_size=3, stride=stride, bias=True)
    self.bn1 = nn.BatchNorm2d(num_channels)

    self.conv2 = nn.Conv2d(in_channels=num_channels, out_channels=num_channels, kernel_size=3, stride=stride, bias=True)
    self.bn2 = nn.BatchNorm2d(num_channels)

    self.conv3 = nn.Conv2d(in_channels=num_channels, out_channels=num_channels, kernel_size=1, stride=stride, bias=True)
    self.bn3 = nn.BatchNorm2d(num_channels)

    self.relu = nn.ReLU()
    self.fc = nn.Linear(in_features=num_channels, out_features=num_classes)
    self.stride = stride
    self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
  
  def forward(self, x):
    out = self.conv1(x)
    out = self.bn1(out)
    out = self.relu(out)

    out = self.conv2(out)
    out = self.bn2(out)
    out = self.relu(out)

    out = self.conv3(out)
    out = self.bn3(out)
    out = self.relu(out)
    out = self.avgpool(out)

    out = out.view(out.size(0), -1)
    out = self.fc(out)

    return out
  
  def compute_loss(self, logits, target):
    loss = F.cross_entropy(logits, target)
    return loss


In [15]:
# Sanity checks for dataset and network
img_dataset = ImageDataset(224, 224, image_names_train, image_labels_train, labels_dict)
img_dataset_validation = ImageDatasetValidation(224, 224, image_names_validation, image_labels_validation, labels_dict)
img_dataset_test = ImageDatasetTest(224, 224, image_names_test, image_labels_test, labels_dict)
image, target = img_dataset.__getitem__(0)
image_val, target_val = img_dataset_validation.__getitem__(0)
image_test, target_test = img_dataset_test.__getitem__(0)
print(image.shape, target)
print(image_val.shape, target_val)
print(image_test.shape, target_test)
conv_net = ConvNetwork(2, 32, 1000)
a = conv_net.forward(torch.rand((1, 3, 224, 224)))

torch.Size([3, 224, 224]) 1
torch.Size([3, 224, 224]) 0
torch.Size([3, 224, 224]) 0


In [16]:
# LR Scheduler
class CosineAnnealingWarmup(_LRScheduler):
    def __init__(
            self,
            optimizer: torch.optim.Optimizer,
            min_lr: float = 0.00001,
            warmup_steps: int = 0,
            decay_steps: int = 100,
            last_epoch: int = -1,
    ):
        self.warmup_steps = warmup_steps
        self.decay_steps = decay_steps
        self.min_lr = min_lr
        self.last_epoch = last_epoch
        super().__init__(optimizer, last_epoch)

    def get_lr(self):
        if self.last_epoch < self.warmup_steps:
            mult = self.last_epoch / self.warmup_steps
        else:
            mult = 0.5 * (
                    1 + cos(pi * (self.last_epoch - self.warmup_steps) / self.decay_steps)
            )
        return [
            self.min_lr + (base_lr - self.min_lr) * mult for base_lr in self.base_lrs
        ]

In [17]:
# Data Module and dataset related stuff for lightning training
import pytorch_lightning as pl
import os
import torch


class BatchCollator:
    """
    From a list of samples from the dataset,
    returns the batched images and targets.
    This should be passed to the DataLoader
    """
    def __init__(self):
        super().__init__()

    def __call__(self, batch):
        transposed_batch = list(zip(*[i for i in batch]))
        images = transposed_batch[0]
        targets = transposed_batch[1]
        images = torch.stack(list(images), dim=0)
        targets = torch.Tensor(targets).long()
        return images, targets


def make_data_loader(dataset, phase, batch_size, num_workers):
    images_per_gpu = batch_size

    collator = BatchCollator()
    data_loader = torch.utils.data.DataLoader(
        dataset,
        num_workers=num_workers,
        batch_size=images_per_gpu,
        collate_fn=collator,
        pin_memory=torch.cuda.is_available(),
        persistent_workers=False,
        shuffle=True,
    )
    return data_loader


class ImgDataModule(pl.LightningDataModule):
    def __init__(self, batch_size):
        super().__init__()
        self.batch_size = batch_size

    def setup(self, stage: str):
        if stage == "fit":
            self.data_train = ImageDataset(224, 224, image_names_train, image_labels_train, labels_dict)
            self.data_validation = ImageDatasetValidation(224, 224, image_names_validation, image_labels_validation, labels_dict)

    def train_dataloader(self):
        return make_data_loader(self.data_train, "train", batch_size=self.batch_size, num_workers=0)
      
    def val_dataloader(self):
        return make_data_loader(self.data_validation, "val", batch_size=self.batch_size, num_workers=0)

In [18]:
# calculating metrics, using sklearn as torchmetrics isn't installed
from sklearn.metrics import accuracy_score

In [19]:
# Lightning Model for training 
class Model(LightningModule):
    def __init__(self, batch_size, stride, num_channels, num_classes) -> None:
        super().__init__()
        self.model = ConvNetwork(stride=stride, 
                                 num_channels=num_channels, 
                                 num_classes=num_classes
                                 )
        self.batch_size = batch_size

    def forward(self, images):
        outputs = self.model.forward(images)
        return outputs

    def process_forward(self, x, targets, phase):
        logits = self.model.forward(x)
        loss = self.model.compute_loss(logits, targets)
        return loss

    def training_step(self, batch, _):
        x, targets = batch
        loss = self.process_forward(x, targets, "train")
        return loss

    def calculate_metrics(self, logits, targets):
        # Apply the sigmoid function to convert logits to probabilities
        probabilities = torch.nn.functional.softmax(logits, dim=1)

        # Convert probabilities to binary predictions using a threshold of 0.5
        predictions = torch.argmax(probabilities, dim=1)

        # Move predictions and targets to the CPU and convert them to NumPy arrays
        predictions_np = predictions.cpu().numpy()
        targets_np = targets.cpu().numpy()

        # Calculate accuracy, precision, and F1 score
        accuracy = accuracy_score(targets_np, predictions_np)

        return accuracy

    def validation_step(self, batch, _):
        x, targets = batch
        loss = self.process_forward(x, targets, "val")
        outputs = self.forward(x)
        accuracy = self.calculate_metrics(outputs, targets)
        self.log("val_loss", loss, on_step=False, on_epoch=True, batch_size=self.batch_size)
        self.log("accuracy", accuracy, on_step=False, on_epoch=True, batch_size=self.batch_size)
        self.log("precision", precision, on_step=False, on_epoch=True, batch_size=self.batch_size)
        self.log("recall", recall, on_step=False, on_epoch=True, batch_size=self.batch_size)
        self.log("f1", f1, on_step=False, on_epoch=True, batch_size=self.batch_size)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=0.001, weight_decay=0.00005)
        scheduler = CosineAnnealingWarmup(optimizer)
        return [optimizer], [scheduler]

# Training script for simply conv model vanilla

In [ ]:
img_data_module = ImgDataModule(batch_size=256)
model_name = "conv_model_vanilla"
# Prepare train dataset and validation dataset
device = torch.device("cuda")

model = Model(batch_size=256, stride=1, num_channels=32, num_classes=len(labels_dict))
model = model.to(device)

output_folder = f"model_outputs/checkpoints/{model_name}"
if not os.path.exists(output_folder + "/"):
    os.makedirs(output_folder + "/")

callbacks = [
    ModelCheckpoint(
        dirpath=output_folder + "/",
        filename="{step}-{val_loss:.3f}-{accuracy:.3f}-{precision:.3f}-{recall:.3f}-{f1:.3f}",
        monitor="val_loss",
        mode="min",
        every_n_epochs=1,
        save_top_k=3,
    ),
]

trainer = Trainer(
    accelerator="gpu",
    max_epochs=25,
    precision=16,
    benchmark=True,
    callbacks=callbacks,
)

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(
    model,
    img_data_module,
)

# Resnet without pretraining 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes):
        super().__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, self.in_channels, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def _make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels),
            )

        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(self.in_channels, out_channels))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

    def compute_loss(self, logits, target):
        loss = F.cross_entropy(logits, target)
        return loss

def resnet34(num_classes):
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes)



# Resnet pretrained

In [20]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

# Your existing BasicBlock, ResNet, and resnet34 classes remain unchanged
def load_pretrained_resnet34(num_classes):
    # Load a pre-trained ResNet34 model from torchvision.models
    model = models.resnet34(pretrained=True)

    # Replace the last fully connected layer to match the number of classes in your dataset
    model.fc = nn.Linear(model.fc.in_features, num_classes)
    return model

num_classes = len(labels_dict)  # Replace this with the number of classes in your dataset

In [21]:
class Model(LightningModule):
    def __init__(self, batch_size, num_classes) -> None:
        super().__init__()
        self.model = load_pretrained_resnet34(num_classes)
        self.batch_size = batch_size

    def forward(self, images):
        outputs = self.model.forward(images)
        return outputs

    def compute_loss(self, logits, target):
        loss = F.cross_entropy(logits, target)
        return loss

    def process_forward(self, x, targets, phase):
        logits = self.model.forward(x)
        loss = self.compute_loss(logits, targets)
        return loss

    def training_step(self, batch, _):
        x, targets = batch
        loss = self.process_forward(x, targets, "train")
        return loss

    def calculate_metrics(self, logits, targets):
        # Apply the sigmoid function to convert logits to probabilities
        probabilities = torch.nn.functional.softmax(logits, dim=1)

        # Convert probabilities to predictions
        predictions = torch.argmax(probabilities, dim=1)

        # Move predictions and targets to the CPU and convert them to NumPy arrays
        predictions_np = predictions.cpu().numpy()
        targets_np = targets.cpu().numpy()

        # Calculate accuracy
        accuracy = accuracy_score(targets_np, predictions_np)
        return accuracy

    def validation_step(self, batch, _):
        x, targets = batch
        loss = self.process_forward(x, targets, "val")
        outputs = self.forward(x)
        accuracy = self.calculate_metrics(outputs, targets)
        self.log("val_loss", loss, on_step=False, on_epoch=True, batch_size=self.batch_size)
        self.log("accuracy", accuracy, on_step=False, on_epoch=True, batch_size=self.batch_size)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=0.00001, weight_decay=0.00005)
        scheduler = CosineAnnealingWarmup(optimizer)
        return [optimizer], [scheduler]

In [22]:
img_data_module = ImgDataModule(batch_size=32)
model_name = "restnet34_pretrained_high_resolution_low_lr"
# Prepare train dataset and validation dataset
device = torch.device("cuda")

model = Model(batch_size=32, num_classes=len(labels_dict))
model = model.to(device)

output_folder = f"model_outputs/checkpoints/{model_name}"
if not os.path.exists(output_folder + "/"):
    os.makedirs(output_folder + "/")

callbacks = [
    ModelCheckpoint(
        dirpath=output_folder + "/",
        filename="{step}-{val_loss:.3f}-{accuracy:.3f}",
        monitor="val_loss",
        mode="min",
        every_n_epochs=1,
        save_top_k=3,
    ),
]

trainer = Trainer(
    accelerator="gpu",
    max_epochs=20,
    precision=16,
    benchmark=True,
    callbacks=callbacks,
)

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


  0%|          | 0.00/83.3M [00:00<?, ?B/s]

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [23]:
trainer.fit(
    model,
    img_data_module,
)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 21.3 M
---------------------------------
21.3 M    Trainable params
0         Non-trainable params
21.3 M    Total params
42.575    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:488: PossibleUserWarning: Your `val_dataloader`'s sampler has shuffling enabled, it is strongly recommended that you turn shuffling off for val/test/predict dataloaders.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


In [24]:
# Test accuracy on test set
model_dict = torch.load("/content/model_outputs/checkpoints/restnet34_pretrained_high_resolution_low_lr/step=2457-val_loss=0.172-accuracy=0.940.ckpt",
                          map_location=device)
model.load_state_dict(model_dict["state_dict"])
model = model.eval()
model = model.to(device)

In [32]:
def convert_logits_to_predictions(logits):
  probabilities = torch.nn.functional.softmax(logits, dim=1)
  # Convert probabilities to predictions
  predictions = torch.argmax(probabilities, dim=1)
  return predictions.cpu()

In [41]:
targets = []
predictions = []

for i in range(img_dataset_test.__len__()):
  image, target = img_dataset_test.__getitem__(i)
  logits = model.forward(image.unsqueeze(0).to(device))
  prediction = convert_logits_to_predictions(logits)
  targets.append(target)
  predictions.append(prediction.item())

In [44]:
# Test accuracy score 94% accuracy on test, quite good
accuracy_score(targets, predictions)

0.9406666666666667